<a href="https://colab.research.google.com/github/adityapatil4141/ML-pipelines-with-zenml/blob/main/ML_pipelines_with_ZenML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pyparsing==2.4.2


In [ ]:
%pip install "zenml[server]"
!zenml integration install sklearn -y

import IPython

IPython.Application.instance().kernel.do_shutdown(restart=True)

NumExpr defaulting to 2 threads.
⠋ Installing integrations...


{'status': 'ok', 'restart': True}

In [ ]:
 NGROK_TOKEN = "2euGDh7rwk5xYF0E4HQ0LOTaPaW_4LURmPq3Se3evh5sZ8cZB"

In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab():

  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.56.3
Creating database tables
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
Creating default workspace 'default' ...
Creating default stack in workspace default...
⠹ Initializing ZenML repository at /content.
Creating default user 'default' ...
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /

In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test():
  digits = load_digits()
  data = digits.images.reshape((len(digits.images),-1))
  x_train,x_test,y_train,y_test = train_test_split(data,digits.target,test_size=0.2,shuffle=False)
  model = SVC(gamma=0.001)
  model.fit(x_train,y_train)
  test_acc = model.score(x_test,y_test)
  print(f"test accuracy {test_acc}")

train_test()

test accuracy 0.9583333333333334


In [ ]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple
@step
def importer() -> Tuple[
    Annotated[np.ndarray, "x_train"],
    Annotated[np.ndarray, "x_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    digits = load_digits()
    data = digits.images.reshape((len(digits.images),-1))
    x_train,x_test,y_train,y_test = train_test_split(data,digits.target, test_size=0.2, shuffle = False)
    return x_train,x_test,y_train,y_test

@step
def svc_trainer(
    x_train: np.ndarray,
    y_train: np.ndarray,
)    -> ClassifierMixin:
    model = SVC(gamma=0.001)
    model.fit(x_train,y_train)
    return model


@step
def evaluator(
    x_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
)    -> float:

    test_acc = model.score(x_test,y_test)
    print(f"Test accuracy: {test_acc}")
    return test_acc


In [ ]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  x_train,x_test,y_train,y_test = importer()
  model = svc_trainer(x_train=x_train,y_train=y_train)
  evaluator(x_test=x_test,y_test=y_test,model=model)

In [ ]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Step importer has started.
Step importer has finished in 4.613s.
Step svc_trainer has started.
Step svc_trainer has finished in 0.950s.
Step evaluator has started.
Test accuracy: 0.9583333333333334
Step evaluator has finished in 1.133s.
Pipeline run has finished in 7.680s.


In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok
    public_url = ngrok.connect(port)
    print(f"\x1b[31mIn colab, use this URL instead: {public_url}!\x1b[om")
    !zenml up --blocking --port {port}
  else:
    !zenml up --port {port}


start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-71ceb778-aab0-4106-920e-170923b1699b


Opening tunnel named: http-8237-71ceb778-aab0-4106-920e-170923b1699b


INFO:pyngrok.process.ngrok:t=2024-04-10T12:42:53+0000 lvl=info msg="no configuration paths supplied"


t=2024-04-10T12:42:53+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-04-10T12:42:53+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

Got token from cookie
INFO:     2401:4900:79f8:3028:44bc:7289:492f:671e:0 - "GET /api/v1/info HTTP/1.1" 200 OK
INFO:     2401:4900:79f8:3028:44bc:7289:492f:671e:0 - "GET /api/v1/workspaces HTTP/1.1" 200 OK
t=2024-04-10T12:44:48+0000 lvl=info msg="join connections" obj=join id=45cbad0fded2 l=127.0.0.1:8237 r=[2401:4900:79f8:3028:44bc:7289:492f:671e]:54584
Got token from cookie
INFO:     2401:4900:79f8:3028:44bc:7289:492f:671e:0 - "GET /api/v1/workspaces/default/pipelines?sort_by=desc:created&logical_operator=and&page=1&size=10 HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2024-04-10T12:44:53+0000 lvl=info msg="join connections" obj=join id=77ac1ae480ef l=127.0.0.1:8237 r=[2401:4900:79f8:3028:44bc:7289:492f:671e]:54584


t=2024-04-10T12:44:53+0000 lvl=info msg="join connections" obj=join id=77ac1ae480ef l=127.0.0.1:8237 r=[2401:4900:79f8:3028:44bc:7289:492f:671e]:54584
Got token from cookie
INFO:     2401:4900:79f8:3028:44bc:7289:492f:671e:0 - "GET /api/v1/workspaces/default/pipelines?sort_by=desc:created&logical_operator=and&page=1&size=10 HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2024-04-10T12:44:59+0000 lvl=info msg="join connections" obj=join id=8196d1eac880 l=127.0.0.1:8237 r=[2401:4900:79f8:3028:44bc:7289:492f:671e]:54584


t=2024-04-10T12:44:59+0000 lvl=info msg="join connections" obj=join id=8196d1eac880 l=127.0.0.1:8237 r=[2401:4900:79f8:3028:44bc:7289:492f:671e]:54584
Got token from cookie
INFO:     2401:4900:79f8:3028:44bc:7289:492f:671e:0 - "GET /api/v1/workspaces/default/pipelines?sort_by=desc:created&logical_operator=and&page=1&size=10 HTTP/1.1" 200 OK
